In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 73.7 MB/s eta 0:00:00


In [8]:
import pdfplumber
import json
import re

# Path to your PDF file
pdf_path = '/content/drive/MyDrive/project_ir/Depressive Disorders.pdf'

# Function to parse PDF into hierarchical JSON using regex
def parse_pdf_to_json_with_regex(pdf_path):
    json_structure = {}
    section_stack = [json_structure]

    # Define regex patterns for different levels in the text
    patterns = [
        (r"^\d+\s+", 1),       # Matches sections like "1 "
        (r"^\d+\.\d+\s+", 2),   # Matches subsections like "1.1 "
        (r"^[A-Z]\s+", 3),      # Matches items like "A ", "B "
        (r"^[A-Z]\.\d+\s+", 4), # Matches sub-items like "A.1 ", "B.2 "
    ]

    with pdfplumber.open(pdf_path) as pdf:
        # Iterate through each page
        for page in pdf.pages:
            text = page.extract_text()

            # Process each line individually
            for line in text.splitlines():
                line = line.strip()
                matched = False

                # Check each pattern to determine its hierarchy level
                for pattern, level in patterns:
                    if re.match(pattern, line):
                        content = re.sub(pattern, "", line)
                        key = f"{level}.{len(section_stack[-1]) + 1}"

                        # Navigate to the correct depth
                        while len(section_stack) > level:
                            section_stack.pop()

                        # Add content to JSON structure at the appropriate level
                        section_stack[-1][key] = content

                        # Push new level if needed
                        if isinstance(section_stack[-1][key], dict):
                            section_stack.append(section_stack[-1][key])

                        matched = True
                        break

                # Handle lines that don't match any known pattern
                if not matched:
                    section_stack[-1][f"{len(section_stack[-1]) + 1}"] = line

    return json_structure

# Convert parsed structure to JSON
hierarchical_json = parse_pdf_to_json_with_regex(pdf_path)
print(json.dumps(hierarchical_json, indent=2))


{
  "1": "Depressive",
  "2": "Disorders",
  "3": "Depressive disorders",
  "4": "include disruptive mood dysregulation disorder, major",
  "5": "depressive disorder (including major depressive episode), persistent depressive disorder",
  "6": "(dysthymia), premenstrual dysphoric disorder, substance/medication-induced depres-",
  "7": "sive disorder, depressive disorder due to another medical condition, other specified de-",
  "8": "pressive disorder, and unspecified depressive disorder. Unlike in DSM-IV, this chapter",
  "9": "\u201cDepressive Disorders\u201d has been separated from the previous chapter \u201cBipolar and Re-",
  "10": "lated Disorders.\u201d The common feature of all of these disorders is the presence of sad,",
  "11": "empty, or irritable mood, accompanied by somatic and cognitive changes that signifi-",
  "12": "cantly affect the individual\u2019s capacity to function. What differs among them are issues of",
  "13": "duration, timing, or presumed etiology.",
  "14":